# Shopee Code League 5: Logistics
### Data Analytics

### Task
Identify all the orders that are considered late depending on the Service Level Agreements (SLA) with our Logistics Provider.

For the purpose of this question, assume that all deliveries are considered successful by the second attempt.
[Link to kaggle](https://www.kaggle.com/c/open-shopee-code-league-logistic/overview)   

#### 1st approach using business calendar

In [1]:
#Importing libraries
import pandas as pd
import numpy as np
from datetime import datetime

## 1 Data preprocess

### Load datasets

In [10]:
# dataset that contains the order & delivery info
# Note due the file size exceeded 100MB, this file not in github repo folder.
order = pd.read_csv('../../gitignore_largefile/delivery_orders_march.csv')
order.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,1583419016,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,1583318305,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


In [3]:
order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3176313 entries, 0 to 3176312
Data columns (total 6 columns):
 #   Column               Dtype  
---  ------               -----  
 0   orderid              int64  
 1   pick                 int64  
 2   1st_deliver_attempt  float64
 3   2nd_deliver_attempt  float64
 4   buyeraddress         object 
 5   selleraddress        object 
dtypes: float64(2), int64(2), object(2)
memory usage: 145.4+ MB


In [11]:
# dataset that contains the SLA based on seller to buyer location
sla = pd.read_excel('./open-shopee-code-league-logistic/SLA_matrix.xlsx')
sla

,1st Attempt SLA\n(Working Days),Unnamed: 1,Destination (Buyer),Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,Metro Manila,Luzon,Visayas,Mindanao
1,Origin\n(Seller),Metro Manila,3 working days,5 working days,7 working days,7 working days
2,NaN,Luzon,5 working days,5 working days,7 working days,7 working days
3,NaN,Visayas,7 working days,7 working days,7 working days,7 working days
4,NaN,Mindanao,7 working days,7 working days,7 working days,7 working days
5,NaN,NaN,NaN,NaN,NaN,NaN
6,"Working Days are defined as Mon - Sat, Excludi...",NaN,NaN,NaN,NaN,NaN
7,SLA calculation begins from the next day after...,NaN,NaN,NaN,NaN,NaN
8,2nd Attempt must be no later than 3 working da...,NaN,NaN,NaN,NaN,NaN


Compile the buyer and seller location from their `address` column

In [19]:
# get the location using this 
order['selleraddress'][0].split()[-1].lower()

'manila'

In [29]:
# get the location
order['seller_loc'] = [address.split()[-1].lower() for address in order['selleraddress']]
order['buyeraddress'] = [address.split()[-1].lower() for address in order['buyeraddress']]

order['from_to'] = [i + " " + j for i,j in zip(order['seller_loc'], order['buyeraddress'])]

In [30]:
# check the unique combination for from_to
order['from_to'].unique()

array(['manila manila', 'manila luzon', 'manila visayas',
       'manila mindanao', 'luzon luzon'], dtype=object)

Based on the unique combination set, compile a dictionaries for their SLA

In [35]:
sla_dict= {'manila manila': 3,
       'manila luzon': 5,
       'manila visayas': 7,
       'manila mindanao': 7,
       'luzon luzon': 5}

In [38]:
# combine the sla for each row based on the sla dict_dict
sla_dict[order['from_to'][0]]

3

In [39]:
order.columns

Index(['orderid', 'pick', '1st_deliver_attempt', '2nd_deliver_attempt',
       'buyeraddress', 'selleraddress', 'seller_loc', 'from_to'],
      dtype='object')

In [40]:
order['sla'] = [sla_dict[row] for row in order['from_to']]

# get the desired column from order dataframe
order = order[['orderid', 'pick', '1st_deliver_attempt', '2nd_deliver_attempt','sla']]
order.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,sla
0,2215676524,1583138397,1.583385e+09,NaN,3
1,2219624609,1583309968,1.583463e+09,1.583799e+09,3
2,2220979489,1583306434,1.583460e+09,NaN,3
3,2221066352,1583419016,1.583556e+09,NaN,3
4,2222478803,1583318305,1.583480e+09,NaN,5


## 2 Data analysis

In [41]:
! pip install business_calendar

In [42]:
from business_calendar import Calendar, MO, TU, WE, TH, FR, SA

In [44]:
timestamp = 1583137548
datetime.fromtimestamp(timestamp)

datetime.datetime(2020, 3, 2, 16, 25, 48)

In [59]:
# creat the calendar
# Add holiday 2020/1/1 and 2020/12/31 to avoid warning
#Class that represents a calendar with work and rest days, as well as
#holidays (which of course are rest days).

cal = Calendar(workdays = [MO, TU, WE, TH, FR, SA], 
        holidays = [datetime(2020,1,1),
                    datetime(2020, 3, 25),
                    datetime(2020, 3, 30),
                    datetime(2020, 3, 31),
                    datetime(2020, 12, 31)])

time_zone = 8*3600  # 8 hours

In [109]:
cal.addbusdays(datetime.fromtimestamp(order['1st_deliver_attempt'][1]), int(order['sla'][1])).date()

datetime.date(2020, 3, 10)

In [56]:
datetime.fromtimestamp(order['1st_deliver_attempt'][0]).date()

datetime.date(2020, 3, 5)

In [127]:
is_late = {}

for idx, row in order.iterrows():
    #print(row['orderid'])
    # add the time_zone
    start_time = datetime.fromtimestamp(float(row['pick']) + time_zone)
    first_attempt = datetime.fromtimestamp(float(row['1st_deliver_attempt']) + time_zone)
    
    # check if the first attempt is late or not, will return true if late
    is_delay = first_attempt.date() > cal.addbusdays(start_time, int(row['sla'])).date()
    
    # if there is 2nd attempt delivery (i.e. not deliver in 1st attempt)
    if not is_delay and not pd.isna(row['2nd_deliver_attempt']):
        second_attempt = datetime.fromtimestamp(float(row['2nd_deliver_attempt'])+ time_zone)
        is_delay = second_attempt.date() > cal.addbusdays(first_attempt, 3).date()
    #print(is_delay)
    
    # save the results in dict
    is_late[row['orderid']]=int(is_delay)

In [148]:
submission = pd.DataFrame.from_dict(is_late, orient='index', columns = ['is_late']).reset_index()
submission.rename(columns = {'index':'orderid'}, inplace=True)
submission['orderid'] = submission['orderid'].astype('int')

submission.head(10)

,orderid,is_late
0,2215676524,0
1,2219624609,0
2,2220979489,0
3,2221066352,0
4,2222478803,0
5,2222597288,0
6,2222738456,0
7,2224695304,1
8,2224704587,0
9,2225138267,0


In [149]:
#export to csv
submission.to_csv('submission.csv', index = False)